# Static Load Calculations for Snow Slabs

This notebook demonstrates methods for calculating the static load (weight per unit area) of layered snow slabs and their force components on slopes using the `snowpyt_mechparams` package.

## Overview

The static load calculations include:
- **Gravitational load**: Total weight per unit area (vertical force)
- **Shear load**: Force component parallel to the slope surface
- **Normal load**: Force component perpendicular to the slope surface


In [43]:
# Import Libraries
import os
import sys

import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form
from snowpyt_mechparams import density
from snowpyt_mechparams.data_structures import Layer, Slab
from snowpyt_mechparams.static_load import calculate_static_load


## Parse Snowpit Files

Parse CAAML snowpit files to extract real layer data:


In [44]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


## Construct Slab

In [45]:
# C slab if ALL layers above layer of concern have successful density calculations
# Returns dictionary with pit_id -> slab mapping

slabs = {}  # Changed to dictionary to store pit_id -> slab mapping

# Track elimination steps for detailed data funnel analysis
total_pits = len(all_pits)
pits_w_slope_angle = 0
pits_w_layer_of_concern = 0
pits_w_angle_and_layer_of_concern = 0
pits_w_missing_layer_data = 0
pits_w_density_calc_failures = 0
density_calc_failures = 0  # Count individual layer failures
valid_slabs = 0

for pit in all_pits:
    pit_id = pit.core_info.pit_id
    layers = pit.snow_profile.layers

    # Track pits with individual characteristics  
    has_slope_angle = pit.core_info.location.slope_angle is not None
    has_layer_of_concern = any(getattr(layer, 'layer_of_concern', False) for layer in layers)

    if has_slope_angle:
        pits_w_slope_angle += 1
    if has_layer_of_concern:
        pits_w_layer_of_concern += 1

    # Only proceed if pit has BOTH slope angle and layer of concern
    if has_slope_angle and has_layer_of_concern:
        pits_w_angle_and_layer_of_concern += 1
        slab_layers = []
        all_layers_valid = True  # Track if all layers are successfully processed

        for layer in layers:
            if layer.layer_of_concern:
                break # Stop at layer of concern

            layer_hardness = layer.hardness
            layer_geldsetzer_grain_form = convert_grain_form(layer.grain_form_primary)

            # Reject entire pit if any layer has missing required data
            if layer_hardness is None or layer_geldsetzer_grain_form is None:
                all_layers_valid = False
                break # Stop processing this pit

            try:
                layer_density = density.calculate_density(
                    method='geldsetzer',
                    hand_hardness=layer_hardness,
                    grain_form=layer_geldsetzer_grain_form
                )
                # Handle thickness - extract value and convert from string to float
                layer_thickness = layer.thickness
                if isinstance(layer_thickness, list):
                    # First element is thickness as string, second is units
                    layer_thickness = float(layer_thickness[0]) if layer_thickness and layer_thickness[0] else 0.0
                else:
                    layer_thickness = float(layer_thickness) if layer_thickness else 0.0

                # Create layer object if density calculation succeeded
                layer_obj = Layer(thickness=layer_thickness, density=layer_density)
                slab_layers.append(layer_obj)
            except Exception as e:
                density_calc_failures += 1
                all_layers_valid = False
                break # Reject entire pit if any density calculation fails

        # Only create slab if ALL layers above layer of concern were successfully processed
        if all_layers_valid and slab_layers:
            # Handle slope angle - extract value and convert from string to float if needed
            slope_angle_raw = pit.core_info.location.slope_angle
            if isinstance(slope_angle_raw, list):
                # First element is the value (possibly as string), handle conversion
                slope_angle = float(slope_angle_raw[0]) if slope_angle_raw and slope_angle_raw[0] else 0.0
            else:
                slope_angle = float(slope_angle_raw) if slope_angle_raw else 0.0

            slab = Slab(layers=slab_layers, angle=slope_angle)
            slabs[pit_id] = slab  # Store slab with pit_id as key
            valid_slabs += 1
        else:
            # Track different types of failures
            if not slab_layers:
                pits_w_missing_layer_data += 1
            else:
                pits_w_density_calc_failures += 1

print(f"Found {len(slabs)} valid slabs from {pits_w_angle_and_layer_of_concern} candidate pits")
print(f"Pits rejected due to missing layer data: {pits_w_missing_layer_data}")
print(f"Pits rejected due to density calculation failures: {pits_w_density_calc_failures}") 
print(f"Total individual density calculation failures: {density_calc_failures}")


Found 10482 valid slabs from 33279 candidate pits
Pits rejected due to missing layer data: 12274
Pits rejected due to density calculation failures: 10523
Total individual density calculation failures: 9992


## Static Load Calculations



In [46]:
# Implement load calculation for valid slabs
import math

# Store results for analysis
static_load_results = {}

for pit_id, slab in slabs.items():
    # Convert slope angle from degrees to radians
    slope_angle_rad = math.radians(slab.angle)
    slab_thickness = slab.total_thickness

    # Calculate static load components
    gravitational_load, shear_load, normal_load = calculate_static_load(slab, slope_angle_rad)

    # Store results
    static_load_results[pit_id] = {
        'number_of_slab_layers': len(slab.layers),
        'slab_thickness': slab_thickness,
        'gravitational_load': gravitational_load,  # N/m²
        'shear_load': shear_load,                  # N/m²
        'normal_load': normal_load,                # N/m²
        'slope_angle': slab.angle,                 # degrees
        'num_layers': len(slab.layers)
    }

print(f"Calculated static loads for {len(static_load_results)} slabs")

satic_load_results_df = pd.DataFrame(static_load_results)


Calculated static loads for 10482 slabs


In [47]:
# Detailed Data Funnel - Pit Elimination Steps

print("=" * 60)
print("DATA FUNNEL: Pit Elimination Steps")
print("=" * 60)

print(f"1. Total pits processed: {total_pits:,}")

print(f"2. Pits with slope angle: {pits_w_slope_angle:,} ({(pits_w_slope_angle/total_pits*100):.1f}%)")
pits_no_slope = total_pits - pits_w_slope_angle
print(f"   └─ Eliminated (no slope angle): {pits_no_slope:,}")

print(f"3. Pits with layer of concern: {pits_w_layer_of_concern:,} ({(pits_w_layer_of_concern/total_pits*100):.1f}%)")
pits_no_layer_concern = total_pits - pits_w_layer_of_concern  
print(f"   └─ Eliminated (no layer of concern): {pits_no_layer_concern:,}")

print(f"4. Pits with BOTH slope angle AND layer of concern: {pits_w_angle_and_layer_of_concern:,} ({(pits_w_angle_and_layer_of_concern/total_pits*100):.1f}%)")
pits_missing_both = total_pits - pits_w_angle_and_layer_of_concern
print(f"   └─ Eliminated (missing slope angle OR layer of concern): {pits_missing_both:,}")

print(f"5. Pits rejected due to missing layer data: {pits_w_missing_layer_data:,}")
print(f"   └─ Eliminated (incomplete hardness/grain form data): {pits_w_missing_layer_data:,}")

print(f"6. Pits rejected due to density calculation failures: {pits_w_density_calc_failures:,}")  
print(f"   └─ Eliminated (density calculation failed for ≥1 layer): {pits_w_density_calc_failures:,}")

print(f"7. FINAL: Valid slabs for analysis: {len(slabs):,} ({(len(slabs)/total_pits*100):.1f}%)")

print("\nSUMMARY:")
print(f"• Started with {total_pits:,} snowpit observations")
print(f"• Final dataset: {len(slabs):,} valid slabs ({(len(slabs)/total_pits*100):.1f}%)")
print(f"• Total individual layer density calculation failures: {density_calc_failures:,}")
print(f"• Data retention rate: {(len(slabs)/total_pits*100):.1f}%")

# Summary Stats


DATA FUNNEL: Pit Elimination Steps
1. Total pits processed: 50,278
2. Pits with slope angle: 45,515 (90.5%)
   └─ Eliminated (no slope angle): 4,763
3. Pits with layer of concern: 36,408 (72.4%)
   └─ Eliminated (no layer of concern): 13,870
4. Pits with BOTH slope angle AND layer of concern: 33,279 (66.2%)
   └─ Eliminated (missing slope angle OR layer of concern): 16,999
5. Pits rejected due to missing layer data: 12,274
   └─ Eliminated (incomplete hardness/grain form data): 12,274
6. Pits rejected due to density calculation failures: 10,523
   └─ Eliminated (density calculation failed for ≥1 layer): 10,523
7. FINAL: Valid slabs for analysis: 10,482 (20.8%)

SUMMARY:
• Started with 50,278 snowpit observations
• Final dataset: 10,482 valid slabs (20.8%)
• Total individual layer density calculation failures: 9,992
• Data retention rate: 20.8%
